In [12]:
import requests
import pickle
from sklearn.preprocessing import StandardScaler
import time
import warnings
warnings.filterwarnings("ignore", category=UserWarning)


In [13]:
def get_accelerometer_data():
    # Fetch accelerometer data from Blynk URL
    url = "https://blynk.cloud/external/api/get?token=a0R-kWpg-zJ6EX7r0U67MfkoKd72XTwK&V0"
    response = requests.get(url)
    data = response.text  # Assuming data is in the format "X: 4y: 5Z: 10"

    # Split the string and extract values
    try:
        values = data.replace(" ", "").split("y:")
        accel_x = float(values[0].split(":")[1])  # Extract X value
        accel_y = float(values[1].split("Z:")[0])  # Extract Y value
        accel_z = float(values[1].split("Z:")[1])  # Extract Z value
        return accel_x, accel_y, accel_z
    except (IndexError, ValueError):
        print("Error parsing accelerometer data.")
        return None, None, None


In [14]:
def get_vibration_data():
    # Fetch vibration sensor data from Blynk URL
    url = "https://blynk.cloud/external/api/get?token=a0R-kWpg-zJ6EX7r0U67MfkoKd72XTwK&V1"
    response = requests.get(url)
    
    try:
        vibration = int(response.text)  # Assuming the response is just the integer value
        return float(vibration)  # Convert to float
    except ValueError:
        print("Error parsing vibration sensor data.")
        return None


In [15]:
# Fetch accelerometer data (X, Y, Z)
accel_x, accel_y, accel_z = get_accelerometer_data()

# Fetch vibration sensor data
vibration = get_vibration_data()

# Check if all data was fetched successfully
if None in [accel_x, accel_y, accel_z, vibration]:
    print("Error fetching sensor data. Prediction cannot proceed.")
else:
    # Prepare data for prediction (example: create a feature vector)
    sensor_data = [accel_x, accel_y, accel_z, vibration]
    #print(f"Sensor Data: {sensor_data}")
        # Print the sensor values
    print(f"Accel X: {accel_x}, Accel Y: {accel_y}, Accel Z: {accel_z}, Vibration: {vibration}")

Accel X: 5.0, Accel Y: -2.0, Accel Z: 9.0, Vibration: 1.0


In [16]:
# Function to load the pretrained model and scaler
def load_model():
    try:
        with open('rf_model.pkl', 'rb') as file:
            model = pickle.load(file)
        with open('scaler.pkl', 'rb') as file:
            scaler = pickle.load(file)
        return model, scaler
    except FileNotFoundError as e:
        print(f"Error loading model or scaler: {e}")
        return None, None

In [17]:
# Function to predict landslide using sensor data
def predict_landslide(sensor_data, model, scaler):
    try:
        # Scale the sensor data
        sensor_data_scaled = scaler.transform([sensor_data])
        
        # Predict landslide (1) or no landslide (0)
        prediction = model.predict(sensor_data_scaled)
        
        return "Landslide Detected" if prediction == 1 else "No Landslide Detected"
    except Exception as e:
        print(f"Error during prediction: {e}")
        return "Prediction Error"


In [18]:
# Function to send notification to Blynk
def send_blynk_notification(message):
    url = f"https://blynk.cloud/external/api/notify?token=a0R-kWpg-zJ6EX7r0U67MfkoKd72XTwK&msg={message}"
    try:
        requests.get(url)
    except requests.RequestException as e:
        print(f"Error sending notification: {e}")

In [19]:
# Load model and scaler
model, scaler = load_model()
if model is None or scaler is None:
    print("Failed to load model or scaler. Exiting.")
    exit()

In [20]:
 # Create sensor data array for prediction
sensor_data = [accel_x, accel_y, accel_z, vibration]

# Predict if there's a landslide
prediction = predict_landslide(sensor_data, model, scaler)
print("Prediction:", prediction)

# Send a notification if a landslide is detected
if prediction == "Landslide Detected":
    send_blynk_notification("Warning! Landslide Detected!")

# Wait for 5 seconds before the next check
time.sleep(5)

Prediction: Landslide Detected
